In [1]:
import cv2

In [2]:
import numpy as np
import cv2
import argparse  #从命令行读取数据
from collections import deque  #双端序列 在序列的前后都可以执行添加或者删除操作

cap=cv2.VideoCapture(0) #打开摄像头用来捕捉视频 
                        #VideoCapture(int index, int apiPreference = CAP_ANY)
                        #index 默认自带摄像头0，其他的外接摄像头一般是1

pts = deque(maxlen=64) #创建一个最长长度为64的双向序列

Lower_green = np.array([110,50,50]) #创建两个一维array
Upper_green = np.array([130,255,255])

while True:
    ret, img=cap.read() #读取摄像头传入的每一帧图像到img中 是三维矩阵
                        #ret是布尔值，如果读取帧是正确的则返回True ， 如果文件读取到结尾，她的值就返回为False
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #将读取到的BGR图像转化为HSV格式
    
    kernel=np.ones((5,5),np.uint8) #创建一个5 * 5的矩阵 矩阵中每个元素都是1，数据类型是uint8
    
    mask=cv2.inRange(hsv,Lower_green,Upper_green) #二值化图像，二值化去间是之前定义的Lower_green Upper_green
    
    mask = cv2.erode(mask, kernel, iterations=2) #腐蚀二值化后的图像 内核时之前创建的 kernel
    
    mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel) #对图像进行开运算 其实就是先腐蚀后膨胀的过程。
                                                    #开运算可以用来消除小物体，在纤细处分离物体，并且在平滑较大物体的边界的同时不明显改变其面积。
    
    #mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)#闭运算
  
    mask = cv2.dilate(mask, kernel, iterations=1)#对图像进行再次膨胀 迭代一次
  
    res=cv2.bitwise_and(img,img,mask=mask)#bitwise_and是对二进制数据进行“与”操作，
                                          #即对图像（灰度图像或彩色图像均可）每个像素值进行二进制“与”操作
                                          #1&1=1，1&0=0，0&1=0，0&0=0
  
    cnts,heir=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:] #轮廓检测算子
                                            #轮廓的检索模式：CV_RETR_EXTERNAL：只检测最外围轮廓，包含在外围轮廓内的内围轮廓被忽略；
                                            #轮廓的近似方法：仅保存轮廓的拐点信息，把所有轮廓拐点处的点保存入contours向量内，拐点与拐点之间直线段上的信息点不予保留；
    center = None

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        if radius > 5:
            cv2.circle(img, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)

    pts.appendleft(center)
    for i in range (1,len(pts)):
        if pts[i-1]is None or pts[i] is None:
            continue
        thick = int(np.sqrt(len(pts) / float(i + 1)) * 2.5)
        cv2.line(img, pts[i-1],pts[i],(0,0,225),thick)

    cv2.imshow("Frame", img)
    cv2.imshow("mask",mask)
    cv2.imshow("res",res)

    k=cv2.waitKey(30) & 0xFF
    if k==32:
        break
# cleanup the camera and close any open windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 